In [14]:
# Import necessary libraries
from sqlalchemy import create_engine
import pandas as pd
import numpy as np
import datetime

def get_match_data(start_date,time_now_formatted,devengine):


    # Get historical match data on hard surface between start date and yesterday
    elo_hard = pd.read_sql_query(
        f"Select DISTINCT * From Elo_AllMatches_Hard where Date > '{start_date}' and Date not like '{time_now_formatted}'",
        con=devengine,
    )

    # Get historical match data on clay surface between start date and yesterday
    elo_clay = pd.read_sql_query(
        f"Select DISTINCT * From Elo_AllMatches_Clay where Date > '{start_date}' and Date not like '{time_now_formatted}'",
        con=devengine,
    )

    #Get today's matches on hard surface that haven't yet been resulted
    elo_data_hard =pd.read_sql_query(
        f"Select DISTINCT * From Elo_AllMatches_Hard where Date like '{time_now_formatted}' and resulted like 'False'",
        con=devengine,)

    #Get today's matches on clay surface that haven't yet been resulted
    elo_data_clay =pd.read_sql_query(
        f"Select DISTINCT * From Elo_AllMatches_Clay where Date like '{time_now_formatted}' and resulted like 'False'",
        con=devengine,)

    return elo_hard, elo_clay, elo_data_hard, elo_data_clay

# Connect to SQLite database using SQLAlchemy's create_engine
devengine = create_engine("sqlite:///C:/Git/tennis_atp/database/bets_sqllite.db")
# Get current date and time
time_now = datetime.datetime.now()

# Format current date as string in YYYY-MM-DD format
time_now_formatted = time_now.strftime("%Y-%m-%d")

# Get the start date two years ago from today
today =time_now
two_years_ago = (today - datetime.timedelta(days=365*2)).strftime("%Y-%m-%d")

elo_hard, elo_clay, elo_data_hard, elo_data_clay = get_match_data(two_years_ago,time_now_formatted,devengine)

def get_player_record(player, opponent_rank, history, range_low,range_high,auto):
    if auto:
        opponent_rank_low = opponent_rank - range_low
        opponent_rank_high = opponent_rank + range_high
    else:
        opponent_rank_low = range_low
        opponent_rank_high = range_high

    player_history = history[
        (
            (history["Fav"] == player)
            & (
                (history["Dog_Rank"] > opponent_rank_low)
                & (history["Dog_Rank"] < opponent_rank_high)
            )
        )
        | (
            (history["Dog"] == player)
            & (
                (history["Fav_Rank"] > opponent_rank_low)
                & (history["Fav_Rank"] < opponent_rank_high)
            )
        )
    ]
    if player_history.empty == False:
        result = float(
            len(player_history[player_history["Winner"] == player])
            / len(player_history)
        )
        return result, len(player_history)
    else:
        return 0, 0

def get_filtered_data(elo_data, elo):
    result_df = pd.DataFrame()
    for _, row in elo_data.sort_values(by="Time").iterrows():
        low_limit = 50
        high_limit = 50

        fav_percent, games = get_player_record(row.Fav, row.Dog_Rank, elo_hard, low_limit, high_limit, True)
        count = 0
        while games < 10 and count < 200:
            count = count + 1
            low_limit = low_limit + 10
            high_limit = high_limit + 10
            fav_percent, games = get_player_record(row.Fav, row.Dog_Rank, elo_hard, low_limit, high_limit, True)

        low_limit = 50
        high_limit = 50
        dog_percent, games2 = get_player_record(row.Dog, row.Fav_Rank, elo_hard, low_limit, high_limit, True)
        count = 0
        while games2 < 10 and count < 200:
            count = count + 1
            low_limit = low_limit + 10
            high_limit = high_limit + 10
            dog_percent, games2 = get_player_record(row.Dog, row.Fav_Rank, elo_hard, low_limit, high_limit, True)

        if games > 4 and games2 > 4:
            temp_df = pd.DataFrame(
                {
                    "Time": [row.Time],
                    "Fav_Odds": [row.Fav_Odds],
                    "Dog_Odds": [row.Dog_Odds],
                    "Fav": [row.Fav],
                    "Elo_Fav": [row.Elo_Fav],
                    "Fav_Record": ["{:.0%}".format(fav_percent)],
                    "Fav_Games": [games],
                    "Dog": [row.Dog],
                    "Dog_Odds": [row.Dog_Odds],
                    "Dog_Record": ["{:.0%}".format(dog_percent)],
                    "Dog_Games": [games2],
                    "fav_percent": [fav_percent],
                    "dog_percent": [dog_percent],
                    "Sex": [row.Sex],
                }
            )
            result_df = pd.concat([result_df, temp_df])
    return result_df

results_hard=get_filtered_data(elo_data_hard,elo_hard)
results_clay=get_filtered_data(elo_data_clay,elo_clay)
def process_serving_data(result_df):
    try:
        # Try to read the 'servers_today.xlsx' file
        serving = pd.read_csv('mensserving.csv')

        # Try to read the 'servers_today_womens.xlsx' file
        serving_womens = pd.read_csv('womensserving.csv')

        # If 'serving_womens' dataframe is not empty, concatenate with 'serving' dataframe
        if serving_womens.empty == False:
            serving = pd.concat([serving, serving_womens])
        else:
            serving = serving

        # Drop the 'Time' column from the 'serving' dataframe
        #serving = serving.drop(columns='Time')
    except FileNotFoundError as e:
        # If either of the excel files is not found, print an error message and set serving to None
        print("The required excel file could not be found.")
        print("Error:", e)
        serving = None

    if serving is not None:
        try:
            # Try to merge the 'result_df' and 'serving' dataframes on the 'Fav' and 'Dog' columns
            result = pd.merge(result_df, serving, how='left', left_on=['Fav'], right_on=['Name'])
            result = pd.merge(result,serving, how='left', left_on=['Dog'], right_on=['Name'])
            result.rename(
        columns={
            "Service Games Won_x": "Fav_Serve%",
            "Service Games Won_y": "Dog_Serve%",
            "Return Games Won_x": "Fav_Return%",
            "Return Games Won_y": "Dog_Return%",
        },
        inplace=True,)
            result.drop(columns=['Name_x','Name_y'],inplace=True)
    

            # Set 'final_hard' to the 'result' dataframe
            result_serving = result
        except Exception as e:
            # If an error occurs during merging, print an error message and set both 'result' and 'final_hard' to None
            print("Error occured while merging the dataframes.")
            print("Error:", e)
            result = None
            result_serving = None
    else:
        result = None
        result_serving = None
        
    return result, result_serving


_,serving_hard=process_serving_data(results_hard)
_,serving_clay=process_serving_data(results_clay)

elo_hardxx = pd.read_sql_query(
    f"Select DISTINCT * From Elo_AllMatches_Hard where Date > '2022-01-01' and Date not like '{time_now_formatted}' --and Fav_odds>1.2 and Fav_odds<1.4 ",
    con=devengine,)

data_concat = pd.DataFrame(columns=['Date', 'Player', 'Odds', 'Win/Loss'])
for dataset_type in [('Winner', 'Win'), ('Loser', 'Loss')]:
    df = elo_hard[['Date', dataset_type[0], f"{dataset_type[0]}_Odds"]].copy()
    df['Player'] = df[dataset_type[0]]
    df['Odds'] = df[f"{dataset_type[0]}_Odds"]
    df['Win/Loss'] = dataset_type[1]
    df.drop(columns=[dataset_type[0], f"{dataset_type[0]}_Odds"], inplace=True)
    data_concat = pd.concat([data_concat, df])
data_concat = data_concat.sort_index()
data_concat['Odds'] = data_concat.Odds.astype(float)

def analyse_matchups(result_df,data_concat):
    for _,matchup in result_df.iterrows():
        player1=matchup.Fav
        player2=matchup.Dog
        player1_odds=float(matchup.Fav_Odds)
        player1_odds_hi=player1_odds+0.2
        player1_odds_lo=1
        player2_odds=float(matchup.Dog_Odds)
        player2_odds_hi=player2_odds+0.5
        player2_odds_lo=2
        player1=data_concat[data_concat['Player']==player1].copy()
        player2=data_concat[data_concat['Player']==player2].copy()
        player2=player2[(player2['Odds']>player2_odds_lo)&(player2['Odds']<player2_odds_hi)]
        if len(player2)>0:
            winperc2=len(player2[player2['Win/Loss']=='Win'])/len(player2)
        else:
            winperc2=0
        player1=player1[(player1['Odds']>player1_odds_lo)&(player1['Odds']<player1_odds_hi)]
        if len(player1)>0:
            winperc1=len(player1[player1['Win/Loss']=='Win'])/len(player1)
        else:
            winperc1=0
        if len(player1)>5 and len(player2)>3:
            print(matchup.Time,f"{matchup.Fav} ({round(player1_odds_lo,2)}-->{round(player1_odds_hi,2)})",f"{matchup.Dog} ({round(player2_odds_lo,2)}-->{round(player2_odds_hi,2)})")
            print(len(player1),winperc1,len(player2),winperc2)

analyse_matchups(serving_hard,data_concat)
final_hard=serving_hard
final_clay=serving_clay

In [15]:
elo_data_hard

,Surface,Date,Sex,Winner,Loser,Winner_Odds,Loser_Odds,Resulted,Time,Elo_Fav,...,Elo_Fav_Elo,Elo_Dog_Elo,Elo_Fav_Rank,Elo_Dog_Rank,Fav,Dog,Fav_Odds,Dog_Odds,Fav_Rank,Dog_Rank
0,Hard,2023-03-18,Womens,Maria Sakkari,Aryna Sabalenka,3.33,1.34,False,08:00,Aryna Sabalenka,...,1992.809760,1854.521841,2.0,7.0,Aryna Sabalenka,Maria Sakkari,1.34,3.33,2.0,7.0
1,Hard,2023-03-18,Mens,Mikael Ymer,Alexei Popyrin,1.58,2.30,False,04:00,Mikael Ymer,...,1711.553783,1634.285858,57.0,101.0,Mikael Ymer,Alexei Popyrin,1.58,2.30,57.0,101.0
2,Hard,2023-03-18,Mens,Roman Safiullin,Nuno Borges,1.83,1.91,False,04:00,Nuno Borges,...,1706.209543,1624.970448,80.0,113.0,Roman Safiullin,Nuno Borges,1.83,1.91,113.0,80.0
3,Hard,2023-03-18,Mens,Thanasi Kokkinakis,Jan Lennard Struff,1.50,2.48,False,04:00,Thanasi Kokkinakis,...,1793.147772,1791.298621,94.0,134.0,Thanasi Kokkinakis,Jan Lennard Struff,1.50,2.48,94.0,134.0
4,Hard,2023-03-18,Mens,Zhizhen Zhang,Alexander Bublik,2.61,1.46,False,05:30,Alexander Bublik,...,1613.839222,1457.197355,46.0,93.0,Alexander Bublik,Zhizhen Zhang,1.46,2.61,46.0,93.0
5,Hard,2023-03-18,Mens,Quentin Halys,Emil Ruusuvuori,2.60,1.46,False,05:30,Quentin Halys,...,1661.687053,1599.671990,81.0,59.0,Emil Ruusuvuori,Quentin Halys,1.46,2.60,59.0,81.0
6,Hard,2023-03-18,Mens,Richard Gasquet,Aleksandar Kovacevic,1.77,1.98,False,05:30,Aleksandar Kovacevic,...,1810.921715,1640.232484,107.0,43.0,Richard Gasquet,Aleksandar Kovacevic,1.77,1.98,43.0,107.0
7,Hard,2023-03-18,Mens,Matteo Berrettini,Aleksandar Vukic,1.18,4.52,False,07:00,Aleksandar Vukic,...,1712.718391,1626.611744,186.0,23.0,Matteo Berrettini,Aleksandar Vukic,1.18,4.52,23.0,186.0
8,Hard,2023-03-18,Mens,Alexander Shevchenko,Marc Andrea Huesler,2.08,1.69,False,11:00,Alexander Shevchenko,...,1592.997674,1504.557367,132.0,51.0,Marc Andrea Huesler,Alexander Shevchenko,1.69,2.08,51.0,132.0
9,Hard,2023-03-18,Womens,Iga Swiatek,Elena Rybakina,1.24,4.09,False,11:00,Iga Swiatek,...,1974.908477,1861.886487,1.0,10.0,Iga Swiatek,Elena Rybakina,1.24,4.09,1.0,10.0


In [16]:
elo_data_hard[['Time','Dog','Winner','Loser','Resulted','Winner_Odds','Dog_Rank','Fav_Rank']][(elo_data_hard['Dog_Rank']<elo_data_hard['Fav_Rank'])&(elo_data_hard['Dog_Rank']<100)].sort_values(by='Time')

,Time,Dog,Winner,Loser,Resulted,Winner_Odds,Dog_Rank,Fav_Rank
2,04:00,Nuno Borges,Roman Safiullin,Nuno Borges,False,1.83,80.0,113.0


In [17]:
final_hard['Fav_Odds']=final_hard['Fav_Odds'].astype(float)
final_hard['Dog_Odds']=final_hard['Dog_Odds'].astype(float)

In [18]:
for _,i in elo_data_hard.iterrows():
    check1=elo_hard[((elo_hard['Winner']==i.Winner)&(elo_hard['Loser']==i.Loser))|((elo_hard['Loser']==i.Winner)&(elo_hard['Winner']==i.Loser))]
    if check1.empty==False:
        for _, x in check1.iterrows():
            print(f"{x.Winner} beat {x.Loser}")

Maria Sakkari beat Aryna Sabalenka
Thanasi Kokkinakis beat Jan Lennard Struff
Elena Rybakina beat Iga Swiatek
Elena Rybakina beat Iga Swiatek


In [19]:
for _,i in elo_data_clay.iterrows():
    check1=elo_clay[((elo_clay['Winner']==i.Winner)&(elo_clay['Loser']==i.Loser))|((elo_clay['Loser']==i.Winner)&(elo_clay['Winner']==i.Loser))]
    if check1.empty==False:
        for _, x in check1.iterrows():
            print(f"{x.Winner} beat {x.Loser}")

Andrea Vavassori beat Camilo Ugo Carabelli


In [20]:
filter1=(final_hard['Sex']=='Womens') & \
(final_hard['dog_percent']>0.0) & (final_hard['dog_percent']<0.5) & (final_hard['fav_percent']<0.9) & (final_hard['fav_percent']>0.1) & (final_hard['Dog_Odds']<2.5) & (((final_hard['dog_percent']>0.0) & (final_hard['dog_percent']<0.1) & (final_hard['fav_percent']<0.5) & 
(final_hard['fav_percent']>0.4)) | ((final_hard['dog_percent']>0.0) & (final_hard['dog_percent']<0.1) & (final_hard['fav_percent']<0.6) & (final_hard['fav_percent']>0.5)) | 
((final_hard['dog_percent']>0.1) & (final_hard['dog_percent']<0.2) & (final_hard['fav_percent']<0.2) & (final_hard['fav_percent']>0.1)) | ((final_hard['dog_percent']>0.1) & (final_hard['dog_percent']<0.2) & (final_hard['fav_percent']<0.4) & (final_hard['fav_percent']>0.3)) | ((final_hard['dog_percent']>0.1) & (final_hard['dog_percent']<0.2) & (final_hard['fav_percent']<0.5) & (final_hard['fav_percent']>0.4)) | ((final_hard['dog_percent']>0.1) & (final_hard['dog_percent']<0.2) & (final_hard['fav_percent']<0.6) & (final_hard['fav_percent']>0.5)) | ((final_hard['dog_percent']>0.2) & (final_hard['dog_percent']<0.3) & (final_hard['fav_percent']<0.5) & (final_hard['fav_percent']>0.4)) | ((final_hard['dog_percent']>0.2) & (final_hard['dog_percent']<0.3) & (final_hard['fav_percent']<0.9) & (final_hard['fav_percent']>0.8)) | ((final_hard['dog_percent']>0.3) & (final_hard['dog_percent']<0.4) & (final_hard['fav_percent']<0.3) & (final_hard['fav_percent']>0.2)) | ((final_hard['dog_percent']>0.4) & (final_hard['dog_percent']<0.5) & (final_hard['fav_percent']<0.2) & (final_hard['fav_percent']>0.1)))
filter2=(final_hard['Sex']=='Womens') & \
           (((final_hard['dog_percent']>0.4) & (final_hard['dog_percent']<0.5) & (final_hard['fav_percent']<0.8) & (final_hard['fav_percent']>0.7)) | 
            ((final_hard['dog_percent']>0.5) & (final_hard['dog_percent']<0.6) & (final_hard['fav_percent']<0.2) & (final_hard['fav_percent']>0.1)) | 
            ((final_hard['dog_percent']>0.5) & (final_hard['dog_percent']<0.6) & (final_hard['fav_percent']<0.3) & (final_hard['fav_percent']>0.2)) | 
            ((final_hard['dog_percent']>0.5) & (final_hard['dog_percent']<0.6) & (final_hard['fav_percent']<0.6) & (final_hard['fav_percent']>0.5)) | 
            ((final_hard['dog_percent']>0.7) & (final_hard['dog_percent']<0.8) & (final_hard['fav_percent']<0.4) & (final_hard['fav_percent']>0.3)) | 
            ((final_hard['dog_percent']>0.7) & (final_hard['dog_percent']<0.8) & (final_hard['fav_percent']<0.7) & (final_hard['fav_percent']>0.6)) | 
            ((final_hard['dog_percent']>0.8) & (final_hard['dog_percent']<0.9) & (final_hard['fav_percent']<0.5) & (final_hard['fav_percent']>0.4)) | 
            ((final_hard['dog_percent']>0.8) & (final_hard['dog_percent']<0.9) & (final_hard['fav_percent']<0.7) & (final_hard['fav_percent']>0.6)) | 
            ((final_hard['dog_percent']>0.9) & (final_hard['dog_percent']<1.0) & (final_hard['fav_percent']<0.6) & (final_hard['fav_percent']>0.5)) | 
            ((final_hard['dog_percent']>0.9) & (final_hard['dog_percent']<1.0) & (final_hard['fav_percent']<0.7) & (final_hard['fav_percent']>0.6))) & \
           (final_hard['Dog_Odds']<2.5)
final_hard[filter1|filter2]

,Time,Fav_Odds,Dog_Odds,Fav,Elo_Fav,Fav_Record,Fav_Games,Dog,Dog_Record,Dog_Games,fav_percent,dog_percent,Sex,Fav_Serve%,Fav_Return%,Dog_Serve%,Dog_Return%


In [21]:
final_hard[
    (final_hard["Sex"] == "Mens")
    & (
        (
            (final_hard["dog_percent"] > 0.0)
            & (final_hard["dog_percent"] < 0.1)
            & (final_hard["fav_percent"] < 0.8)
            & (final_hard["fav_percent"] > 0.7)
            & (final_hard["Dog_Odds"] < 2.5)
        )
        | (
            (final_hard["dog_percent"] > 0.1)
            & (final_hard["dog_percent"] < 0.2)
            & (final_hard["fav_percent"] < 0.8)
            & (final_hard["fav_percent"] > 0.7)
            & (final_hard["Dog_Odds"] < 2.5)
        )
        | (
            (final_hard["dog_percent"] > 0.1)
            & (final_hard["dog_percent"] < 0.2)
            & (final_hard["fav_percent"] < 0.9)
            & (final_hard["fav_percent"] > 0.8)
            & (final_hard["Dog_Odds"] < 2.5)
        )
        | (
            (final_hard["dog_percent"] > 0.3)
            & (final_hard["dog_percent"] < 0.4)
            & (final_hard["fav_percent"] < 0.4)
            & (final_hard["fav_percent"] > 0.3)
            & (final_hard["Dog_Odds"] < 2.5)
        )
        | (
            (final_hard["dog_percent"] > 0.6)
            & (final_hard["dog_percent"] < 0.7)
            & (final_hard["fav_percent"] < 0.3)
            & (final_hard["fav_percent"] > 0.2)
            & (final_hard["Dog_Odds"] < 2.5)
        )
        | (
            (final_hard["dog_percent"] > 0.6)
            & (final_hard["dog_percent"] < 0.7)
            & (final_hard["fav_percent"] < 0.5)
            & (final_hard["fav_percent"] > 0.4)
            & (final_hard["Dog_Odds"] < 2.5)
        )
        | (
            (final_hard["dog_percent"] > 0.7)
            & (final_hard["dog_percent"] < 0.8)
            & (final_hard["fav_percent"] < 0.4)
            & (final_hard["fav_percent"] > 0.3)
            & (final_hard["Dog_Odds"] < 2.5)
        )
        | (
            (final_hard["dog_percent"] > 0.7)
            & (final_hard["dog_percent"] < 0.8)
            & (final_hard["fav_percent"] < 0.5)
            & (final_hard["fav_percent"] > 0.4)
            & (final_hard["Dog_Odds"] < 2.5)
        )
        | (
            (final_hard["dog_percent"] > 0.7)
            & (final_hard["dog_percent"] < 0.8)
            & (final_hard["fav_percent"] < 0.7)
            & (final_hard["fav_percent"] > 0.6)
            & (final_hard["Dog_Odds"] < 2.5)
        )
        | (
            (final_hard["dog_percent"] > 0.9)
            & (final_hard["dog_percent"] < 1.0)
            & (final_hard["fav_percent"] < 0.7)
            & (final_hard["fav_percent"] > 0.6)
            & (final_hard["Dog_Odds"] < 2.5)
        )
    )

]


,Time,Fav_Odds,Dog_Odds,Fav,Elo_Fav,Fav_Record,Fav_Games,Dog,Dog_Record,Dog_Games,fav_percent,dog_percent,Sex,Fav_Serve%,Fav_Return%,Dog_Serve%,Dog_Return%
1,04:00,1.83,1.91,Roman Safiullin,Nuno Borges,62%,13,Nuno Borges,73%,11,0.615385,0.727273,Mens,86.0,19.0,84.0,13.0


In [22]:
#100%
elo_data_hard['Fav_Odds']=elo_data_hard['Fav_Odds'].astype(float)
elo_data_hard[(elo_data_hard['Dog_Rank']>elo_data_hard['Fav_Rank'])&(elo_data_hard['Fav_Odds']<1.2)&(elo_data_hard['Fav_Odds']>1.1)&(elo_data_hard['Dog_Rank']>200)&(elo_data_hard['Fav_Rank']<50)]

,Surface,Date,Sex,Winner,Loser,Winner_Odds,Loser_Odds,Resulted,Time,Elo_Fav,...,Elo_Fav_Elo,Elo_Dog_Elo,Elo_Fav_Rank,Elo_Dog_Rank,Fav,Dog,Fav_Odds,Dog_Odds,Fav_Rank,Dog_Rank


In [23]:
final_hard[final_hard['Sex']=='Womens']#[final_hard['dog_percent']>0.5]#[(final_hard['Fav_Serve%']<final_hard['Dog_Serve%'])&(final_hard['Dog_Return%']>20)&(final_hard['Fav_Return%']<20)&(final_hard['Fav_Serve%']>0)&(final_hard['Dog_Odds']<2.5)]

,Time,Fav_Odds,Dog_Odds,Fav,Elo_Fav,Fav_Record,Fav_Games,Dog,Dog_Record,Dog_Games,fav_percent,dog_percent,Sex,Fav_Serve%,Fav_Return%,Dog_Serve%,Dog_Return%
7,08:00,1.34,3.33,Aryna Sabalenka,Aryna Sabalenka,68%,22,Maria Sakkari,65%,23,0.681818,0.652174,Womens,67.7,36.9,74.8,33.0
9,11:00,1.24,4.09,Iga Swiatek,Iga Swiatek,79%,29,Elena Rybakina,73%,22,0.793103,0.727273,Womens,75.7,49.5,78.2,30.3


In [24]:
final_hard[final_hard['Sex']=='Mens']

,Time,Fav_Odds,Dog_Odds,Fav,Elo_Fav,Fav_Record,Fav_Games,Dog,Dog_Record,Dog_Games,fav_percent,dog_percent,Sex,Fav_Serve%,Fav_Return%,Dog_Serve%,Dog_Return%
0,04:00,1.58,2.30,Mikael Ymer,Mikael Ymer,67%,21,Alexei Popyrin,46%,13,0.666667,0.461538,Mens,74.0,30.0,80.0,12.0
1,04:00,1.83,1.91,Roman Safiullin,Nuno Borges,62%,13,Nuno Borges,73%,11,0.615385,0.727273,Mens,86.0,19.0,84.0,13.0
2,04:00,1.50,2.48,Thanasi Kokkinakis,Thanasi Kokkinakis,90%,10,Jan Lennard Struff,71%,14,0.900000,0.714286,Mens,84.0,14.0,NaN,NaN
3,05:30,1.46,2.61,Alexander Bublik,Alexander Bublik,57%,14,Zhizhen Zhang,30%,10,0.571429,0.300000,Mens,81.0,20.0,83.0,13.0
4,05:30,1.46,2.60,Emil Ruusuvuori,Quentin Halys,50%,14,Quentin Halys,53%,17,0.500000,0.529412,Mens,78.0,25.0,84.0,15.0
5,05:30,1.77,1.98,Richard Gasquet,Aleksandar Kovacevic,40%,10,Aleksandar Kovacevic,50%,10,0.400000,0.500000,Mens,79.0,23.0,85.0,23.0
6,07:00,1.18,4.52,Matteo Berrettini,Aleksandar Vukic,60%,10,Aleksandar Vukic,40%,10,0.600000,0.400000,Mens,88.0,18.0,85.0,13.0
8,11:00,1.69,2.08,Marc Andrea Huesler,Alexander Shevchenko,55%,11,Alexander Shevchenko,42%,12,0.545455,0.416667,Mens,NaN,NaN,57.0,14.0


In [25]:
final_clay

,Time,Fav_Odds,Dog_Odds,Fav,Elo_Fav,Fav_Record,Fav_Games,Dog,Dog_Record,Dog_Games,fav_percent,dog_percent,Sex,Fav_Serve%,Fav_Return%,Dog_Serve%,Dog_Return%
0,01:30,1.51,2.47,Hugo Gaston,Hugo Gaston,60%,10,Riccardo Bonadio,50%,10,0.6,0.500000,Mens,70.0,22.0,0.0,0.0
1,19:00,1.28,3.56,Fabian Marozsan,Fabian Marozsan,50%,10,Nino Serdarusic,31%,13,0.5,0.307692,Mens,0.0,0.0,0.0,0.0
